CELL 1: Install & Import

In [ ]:
# ============================================
# STEP 1: Libraries Install & Import
# ============================================
!pip install pdfplumber python-docx -q

import pdfplumber          # PDF se text nikalne ke liye
import docx                # Word file se text nikalne ke liye
import re                  # Pattern matching (regex)
from sklearn.feature_extraction.text import TfidfVectorizer  # Text → Numbers
from sklearn.metrics.pairwise import cosine_similarity       # Similarity score

print("Ready!")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.1/68.1 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 53.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 55.2 MB/s eta 0:00:00
Ready!


CELL 2: Skills Database

In [ ]:
# ============================================
# STEP 2: Skills Database (Reference List)
# ============================================
# Ye ek SET hai — isme O(1) mein search hota hai
# Iske against resume aur JD dono check honge

SKILLS_DB = {
    # Programming Languages
    "python", "java", "javascript", "typescript", "c++", "c#",
    "ruby", "php", "swift", "kotlin", "go", "rust", "scala", "r",

    # Web Frameworks
    "react", "angular", "vue", "nextjs", "django", "flask",
    "fastapi", "spring boot", "express", "nodejs", "laravel",
    "bootstrap", "tailwind", "jquery", "html", "css",

    # Databases
    "sql", "mysql", "postgresql", "mongodb", "redis", "sqlite",
    "oracle", "cassandra", "elasticsearch", "dynamodb", "firebase",

    # Cloud & DevOps
    "aws", "azure", "gcp", "docker", "kubernetes", "terraform",
    "jenkins", "ci/cd", "linux", "nginx", "ansible", "github actions",
    "heroku", "vercel", "serverless", "microservices",

    # AI / ML / Data
    "machine learning", "deep learning", "nlp", "computer vision",
    "tensorflow", "pytorch", "keras", "scikit-learn", "pandas",
    "numpy", "matplotlib", "opencv", "transformers", "langchain",
    "data science", "data analysis", "power bi", "tableau", "excel",

    # Tools
    "git", "github", "docker", "postman", "jira", "figma",
    "selenium", "rest api", "graphql", "agile", "scrum",

    # Soft Skills
    "communication", "teamwork", "leadership", "problem solving",
}

print(f" Skills Database Ready — {len(SKILLS_DB)} skills loaded")

 Skills Database Ready — 90 skills loaded


CELL 3: Core Functions

In [ ]:
# ============================================
# STEP 3: Core Functions
# ============================================

# ----- FUNCTION 1: PDF se text nikalo -----
def read_pdf(path):
    """PDF file open karke text extract karta hai"""
    text = ""
    with pdfplumber.open(path) as pdf:
        for page in pdf.pages:               # har page loop karo
            t = page.extract_text()
            if t:
                text += t + "\n"
    return text


# ----- FUNCTION 2: DOCX se text nikalo -----
def read_docx(path):
    """Word file ke paragraphs read karta hai"""
    doc = docx.Document(path)
    return "\n".join([p.text for p in doc.paragraphs])


# ----- FUNCTION 3: File type detect karke text nikalo -----
def get_text(path):
    """Extension dekh ke sahi reader call karta hai"""
    if path.endswith(".pdf"):
        return read_pdf(path)
    elif path.endswith(".docx"):
        return read_docx(path)
    else:
        return open(path).read()


# ----- FUNCTION 4: Text mein se skills dhundho -----
def extract_skills(text):
    """
    Text ko lowercase karo,
    phir har skill check karo ki text mein hai ya nahi
    Regex se EXACT word match karte hain (partial nahi)
    """
    text = text.lower()
    found = set()

    # Lambi skills PEHLE check — "machine learning" before "machine"
    for skill in sorted(SKILLS_DB, key=len, reverse=True):
        # \b = word boundary → exact match ensure karta hai
        pattern = r'\b' + re.escape(skill) + r'\b'
        if re.search(pattern, text):
            found.add(skill)

    return found


# ----- FUNCTION 5: TF-IDF + Cosine Similarity -----
def content_similarity(text1, text2):
    """
    Step 1: TF-IDF se dono texts ko number vectors mein badlo
    Step 2: Cosine similarity se angle measure karo
    Result: 0.0 (bilkul alag) to 1.0 (bilkul same)
    """
    vectorizer = TfidfVectorizer(stop_words='english')
    vectors = vectorizer.fit_transform([text1, text2])
    score = cosine_similarity(vectors[0:1], vectors[1:2])[0][0]
    return round(score * 100, 1)   # percentage mein


# ----- FUNCTION 6: Resume sections check karo -----
def check_sections(text):
    """Resume mein standard sections hain ya nahi"""
    text = text.lower()
    sections = {
        "Education"      : bool(re.search(r"education|degree|university|college", text)),
        "Experience"     : bool(re.search(r"experience|work history|employment", text)),
        "Skills"         : bool(re.search(r"skills|technical skills|technologies", text)),
        "Projects"       : bool(re.search(r"projects|project work", text)),
        "Certifications" : bool(re.search(r"certification|certified", text)),
        "Summary"        : bool(re.search(r"summary|objective|profile", text)),
    }
    return sections


print("All functions ready!")

All functions ready!


CELL 4: ATS Score Calculator

In [ ]:
# ============================================
# STEP 4: ATS Score Calculator
# ============================================

def calculate_ats(resume_text, jd_text):
    """
    SCORING FORMULA:
    ────────────────
    ATS Score = Skill Match   × 50%
              + TF-IDF Score  × 30%
              + Section Score × 10%
              + Keyword Score × 10%
    """

    # ── A. SKILL MATCHING (50% weight) ──
    resume_skills = extract_skills(resume_text)
    jd_skills     = extract_skills(jd_text)

    matched = resume_skills & jd_skills        # SET INTERSECTION (common)
    missing = jd_skills - resume_skills        # SET DIFFERENCE (JD mein hai, resume mein nahi)
    extra   = resume_skills - jd_skills        # Bonus skills

    if len(jd_skills) > 0:
        skill_score = (len(matched) / len(jd_skills)) * 100
    else:
        skill_score = 0

    # ── B. CONTENT SIMILARITY (30% weight) ──
    similarity = content_similarity(resume_text, jd_text)

    # ── C. SECTION CHECK (10% weight) ──
    sections = check_sections(resume_text)
    section_score = (sum(sections.values()) / len(sections)) * 100

    # ── D. KEYWORD OVERLAP (10% weight) ──
    jd_words = set(re.findall(r'\b[a-z]{3,}\b', jd_text.lower()))
    res_words = set(re.findall(r'\b[a-z]{3,}\b', resume_text.lower()))
    common_words = jd_words & res_words
    keyword_score = (len(common_words) / len(jd_words)) * 100 if jd_words else 0

    # ── E. FINAL WEIGHTED SCORE ──
    ats_score = (
        skill_score    * 0.50 +
        similarity     * 0.30 +
        section_score  * 0.10 +
        keyword_score  * 0.10
    )
    ats_score = min(round(ats_score, 1), 100)

    return {
        "ats_score"     : ats_score,
        "skill_score"   : round(skill_score, 1),
        "similarity"    : similarity,
        "section_score" : round(section_score, 1),
        "keyword_score" : round(keyword_score, 1),
        "matched"       : matched,
        "missing"       : missing,
        "extra"         : extra,
        "sections"      : sections,
        "jd_skills"     : jd_skills,
        "resume_skills" : resume_skills,
    }


print(" ATS Calculator ready!")

 ATS Calculator ready!


 CELL 5: Print Report Function

In [ ]:
# ============================================
# STEP 5: Report Printer
# ============================================

def print_report(r):
    """Result dictionary leke beautiful report print karo"""

    score = r["ats_score"]
    if   score >= 80: grade = "🟢 EXCELLENT"
    elif score >= 60: grade = "🟡 GOOD"
    elif score >= 40: grade = "🟠 AVERAGE"
    else:             grade = "🔴 POOR"

    # print("=" * 55)
    # print("       🤖 AI RESUME SCREENER — ATS REPORT")
    # print("=" * 55)

    print(f"\n   📊 ATS SCORE : {score} / 100  {grade}\n")

    print("-" * 55)
    print(" SCORE BREAKDOWN:")
    print(f"    Skill Match      : {r['skill_score']}%  (50% weight)")
    print(f"    Content Match    : {r['similarity']}%  (30% weight)")
    print(f"    Resume Sections  : {r['section_score']}%  (10% weight)")
    print(f"    Keyword Overlap  : {r['keyword_score']}%  (10% weight)")

    print("-" * 55)
    print(f" MATCHED SKILLS ({len(r['matched'])} / {len(r['jd_skills'])} JD skills):")
    if r["matched"]:
        print(f"   {', '.join(sorted(r['matched']))}")
    else:
        print("    no skills are matched!")

    print(f"\n MISSING SKILLS ({len(r['missing'])} skills — ye add karo!):")
    if r["missing"]:
        print(f"   {', '.join(sorted(r['missing']))}")
    else:
        print("    every skills are covered!")

    print(f"\n EXTRA SKILLS (are in resume, but not in jd):")
    if r["extra"]:
        print(f"   {', '.join(sorted(r['extra']))}")
    else:
        print("   — None")

    print("-" * 55)
    print(" RESUME SECTIONS:")
    for sec, found in r["sections"].items():
        status = "✅" if found else "❌"
        print(f"   {status} {sec}")

    print("-" * 55)
    print("💡 RECOMMENDATIONS:")
    if r["missing"]:
        top = list(r["missing"])[:5]
        print(f"   🔸 Add these skills: {', '.join(top)}")
    if not r["sections"].get("Projects"):
        print("   🔸 Add a Projects section")
    if not r["sections"].get("Summary"):
        print("   🔸 Add a Summary/Objective section")
    if score >= 80:
        print("   🔸 Great resume! Apply confidently! 🚀")
    print("=" * 55)


print("Report printer ready!")

Report printer ready!


 CELL 6: RUN


In [ ]:
# ============================================
# OPTION A: FILE UPLOAD (PDF / DOCX)
# ============================================
from google.colab import files

# Step 1: Resume upload karo
print(" upload rour resume (PDF or DOCX):")
uploaded = files.upload()
resume_path = list(uploaded.keys())[0]

# Step 2: JD paste karo
jd_text = input("\n Job Description :\n")

# Step 3: Run!
resume_text = get_text(resume_path)
result = calculate_ats(resume_text, jd_text)
print_report(result)

 upload rour resume (PDF or DOCX):


KeyboardInterrupt: 